In [2]:
import re
from dataset import Dataset
from dataset_constants import *

In [3]:
dataset = Dataset(train_path=TRAIN_DATASET_PATH,
                  val_path=VALIDATION_DATASET_PATH, 
                  test_path=TEST_DATASET_PATH)
dataset.build()

In [4]:
X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")

print(X_train[:5])
print(Y_train[:5])
print(len(X_train))
print(len(Y_train))

['Inaasahan na ni Vice President Jejomar Binay na may mga taong... https://t.co/SDytgbWiLh', 'Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA NGA DI STRAIGHT', 'Salamat sa walang sawang suporta ng mga taga makati! Ang Pagbabalik Binay In Makati #OnlyBinayInMakatiSanKaPa https://t.co/iwAOdtZPRE', '@rapplerdotcom putangina mo binay TAKBO PA', 'Binay with selective amnesia, forgetting about the past six years he spent preparing to be president.  #PiliPinasDebates2016']
[0, 1, 0, 1, 0]
10000
10000


In [5]:
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")

print(X_val[:5])
print(Y_val[:5])
print(len(X_val))
print(len(Y_val))

['Escudero denies betraying Poe after meeting with Binay |https://t.co/sKlXTIhHJa - Kare-kare at sinampalukan ang topic. Walang balimbing?', 'Hndi ko makita yung sa one more chance saka kay binay sa fb. Haist.', "Mar Roxas is now addressing the crowd gathered at Pasay City's Ulat sa Barangay 2016  https://t.co/VruZyJ2e2H", '@ImYourBaeMax perfect! Para makaharap ni Duterte ang mga Binay at makatikim ng mura #^%* i#*', '#OnlyBinayPriority4Ps Wag nating hayaan na maloko tayo ng mga pulitikong yan. Kay Binay na tayo']
[0, 1, 0, 0, 0]
4232
4232


In [6]:
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

print(X_test[:5])
print(Y_test[:5])
print(len(X_test))
print(len(Y_test))

['Unshaded votes and votes for Mayor Duterte goes to Mar Roxas according to some reports of ballot tests.  #AyawSaDILAW', '#NoMoreChance https://t.co/msaaUGv0bS', "@itsmanj well there's other good choices like Duterte or Poe. But both of them are still undecided, I think? :( :(", 'Nognog. Pandak. Laki sa hirap. Pero corrupt. Yan si Binay!!!', 'Ex-Binay aide turns tables on Mercado | https://t.co/nyySAo54rL']
[1, 1, 0, 1, 0]
4232
4232


In [7]:
X = dataset.get_features()
Y = dataset.get_labels()
print(len(X))
print(len(Y))
print(X[:5])
print(Y[:5])

18464
18464
['Inaasahan na ni Vice President Jejomar Binay na may mga taong... https://t.co/SDytgbWiLh', 'Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA NGA DI STRAIGHT', 'Salamat sa walang sawang suporta ng mga taga makati! Ang Pagbabalik Binay In Makati #OnlyBinayInMakatiSanKaPa https://t.co/iwAOdtZPRE', '@rapplerdotcom putangina mo binay TAKBO PA', 'Binay with selective amnesia, forgetting about the past six years he spent preparing to be president.  #PiliPinasDebates2016']
[0, 1, 0, 1, 0]


In [9]:
def count_texts_with_urls(texts):
    # Define the regex pattern for URLs
    url_pattern = r'https?://\S+'
    # Count texts containing at least one URL
    count = sum(bool(re.search(url_pattern, text)) for text in texts)
    return count

In [10]:
def remove_urls(X):
    # Define the regex pattern for URLs
    url_pattern = r'https?://\S+'
    # Remove URLs from each text in the list
    return [re.sub(url_pattern, '', text) for text in X]

In [11]:
def remove_usernames(texts):
    # Define the regex pattern for usernames
    username_pattern = r'@\S+'
    
    # Remove usernames from each text in the list
    return [re.sub(username_pattern, '', text) for text in texts]

In [12]:
def count_texts_with_usernames(texts):
    # Regular expression pattern to match usernames
    pattern = r'@\S+'
    count = 0
    
    # Iterate through each text and check for the presence of the pattern
    for text in texts:
        if re.search(pattern, text):
            count += 1
    
    return count

In [11]:
print(count_texts_with_urls(X))
X_url_removed = remove_urls(X)
print(count_texts_with_urls(X_url_removed))

5016
0


In [12]:
print(X_url_removed[:10])

['Inaasahan na ni Vice President Jejomar Binay na may mga taong... ', 'Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA NGA DI STRAIGHT', 'Salamat sa walang sawang suporta ng mga taga makati! Ang Pagbabalik Binay In Makati #OnlyBinayInMakatiSanKaPa ', '@rapplerdotcom putangina mo binay TAKBO PA', 'Binay with selective amnesia, forgetting about the past six years he spent preparing to be president.  #PiliPinasDebates2016', "It doesn't matter whoever won between Duterte &amp; Miriam as President. As long as they finally changed the country Noynoy failed to do so.", 'Nognog? Pero nognog din ang nag malasakit? Wtf? Tangina mo Binay nagpapaawa kapa! Hahahahaha #Nognog ??', '#OnlyB1nay ?? #FB ', 'What Abi Binay said on running for Makati mayor #Halalan2016  ', 'Srsly. How can Binay do away with no tax for those earning PhP 30k and below without any compromises? Nkklk talaga!']


In [13]:
print(count_texts_with_usernames(X_url_removed))
X_username_url_removed = remove_usernames(X_url_removed)
print(count_texts_with_urls(X_username_url_removed))

3950
0


In [14]:
dataset.save_to_file(X_username_url_removed, Y, CLEANED_DATASET_PATH)

Data saved to cleaned_dataset.pkl


In [15]:
X, Y = dataset.load_from_file(CLEANED_DATASET_PATH)

Data loaded from cleaned_dataset.pkl


In [8]:
X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")

print(X_train[:5])
print(Y_train[:5])
print(len(X_train))
print(len(Y_train))

['Inaasahan na ni Vice President Jejomar Binay na may mga taong... https://t.co/SDytgbWiLh', 'Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA NGA DI STRAIGHT', 'Salamat sa walang sawang suporta ng mga taga makati! Ang Pagbabalik Binay In Makati #OnlyBinayInMakatiSanKaPa https://t.co/iwAOdtZPRE', '@rapplerdotcom putangina mo binay TAKBO PA', 'Binay with selective amnesia, forgetting about the past six years he spent preparing to be president.  #PiliPinasDebates2016']
[0, 1, 0, 1, 0]
10000
10000


In [9]:
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")

print(X_val[:5])
print(Y_val[:5])
print(len(X_val))
print(len(Y_val))

['Escudero denies betraying Poe after meeting with Binay |https://t.co/sKlXTIhHJa - Kare-kare at sinampalukan ang topic. Walang balimbing?', 'Hndi ko makita yung sa one more chance saka kay binay sa fb. Haist.', "Mar Roxas is now addressing the crowd gathered at Pasay City's Ulat sa Barangay 2016  https://t.co/VruZyJ2e2H", '@ImYourBaeMax perfect! Para makaharap ni Duterte ang mga Binay at makatikim ng mura #^%* i#*', '#OnlyBinayPriority4Ps Wag nating hayaan na maloko tayo ng mga pulitikong yan. Kay Binay na tayo']
[0, 1, 0, 0, 0]
4232
4232


In [10]:
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

print(X_test[:5])
print(Y_test[:5])
print(len(X_test))
print(len(Y_test))

['Unshaded votes and votes for Mayor Duterte goes to Mar Roxas according to some reports of ballot tests.  #AyawSaDILAW', '#NoMoreChance https://t.co/msaaUGv0bS', "@itsmanj well there's other good choices like Duterte or Poe. But both of them are still undecided, I think? :( :(", 'Nognog. Pandak. Laki sa hirap. Pero corrupt. Yan si Binay!!!', 'Ex-Binay aide turns tables on Mercado | https://t.co/nyySAo54rL']
[1, 1, 0, 1, 0]
4232
4232


In [19]:
print(X[0])
print(Y[0])

Inaasahan na ni Vice President Jejomar Binay na may mga taong... https://t.co/SDytgbWiLh
0


## Clean each set separately

### Train Set

In [7]:
X_train = dataset.X_train
Y_train = dataset.Y_train

In [13]:
print(count_texts_with_urls(X_train))
X_train_url_removed = remove_urls(X_train)
print(count_texts_with_urls(X_train_url_removed))

2685
0


In [14]:
print(count_texts_with_usernames(X_train_url_removed))
X_train_username_url_removed = remove_usernames(X_train_url_removed)
print(count_texts_with_urls(X_train_username_url_removed))

2094
0


In [17]:
dataset.save_to_file(X_train_username_url_removed, Y_train, 'train.pkl')

Data saved to train.pkl


### Validation Set

In [18]:
X_val = dataset.X_val
Y_val = dataset.Y_val

In [19]:
print(count_texts_with_urls(X_val))
X_val_url_removed = remove_urls(X_val)
print(count_texts_with_urls(X_val_url_removed))

1171
0


In [20]:
print(count_texts_with_usernames(X_val_url_removed))
X_val_username_url_removed = remove_usernames(X_val_url_removed)
print(count_texts_with_urls(X_val_username_url_removed))

903
0


In [21]:
dataset.save_to_file(X_val_username_url_removed, Y_val, 'valid.pkl')

Data saved to valid.pkl


### Test Set

In [22]:
X_test = dataset.X_test
Y_test = dataset.Y_test

In [23]:
print(count_texts_with_urls(X_test))
X_test_url_removed = remove_urls(X_test)
print(count_texts_with_urls(X_test_url_removed))

1160
0


In [24]:
print(count_texts_with_usernames(X_test_url_removed))
X_test_username_url_removed = remove_usernames(X_test_url_removed)
print(count_texts_with_urls(X_test_username_url_removed))

953
0


In [25]:
dataset.save_to_file(X_test_username_url_removed, Y_test, 'test.pkl')

Data saved to test.pkl


### Load Datasets

In [26]:
X_train, Y_train = dataset.load_from_file('train.pkl')
print(len(X_train))
print(len(Y_train))

Data loaded from train.pkl
10000
10000


In [27]:
print(X_train[:5])
print(Y_train[:5])

['Inaasahan na ni Vice President Jejomar Binay na may mga taong... ', 'Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA NGA DI STRAIGHT', 'Salamat sa walang sawang suporta ng mga taga makati! Ang Pagbabalik Binay In Makati #OnlyBinayInMakatiSanKaPa ', ' putangina mo binay TAKBO PA', 'Binay with selective amnesia, forgetting about the past six years he spent preparing to be president.  #PiliPinasDebates2016']
[0, 1, 0, 1, 0]


In [28]:
X_valid, Y_valid = dataset.load_from_file('valid.pkl')
print(len(X_valid))
print(len(Y_valid))

Data loaded from valid.pkl
4232
4232


In [30]:
print(X_val[:5])
print(Y_val[:5])

['Escudero denies betraying Poe after meeting with Binay |https://t.co/sKlXTIhHJa - Kare-kare at sinampalukan ang topic. Walang balimbing?', 'Hndi ko makita yung sa one more chance saka kay binay sa fb. Haist.', "Mar Roxas is now addressing the crowd gathered at Pasay City's Ulat sa Barangay 2016  https://t.co/VruZyJ2e2H", '@ImYourBaeMax perfect! Para makaharap ni Duterte ang mga Binay at makatikim ng mura #^%* i#*', '#OnlyBinayPriority4Ps Wag nating hayaan na maloko tayo ng mga pulitikong yan. Kay Binay na tayo']
[0, 1, 0, 0, 0]


In [29]:
X_test, Y_test = dataset.load_from_file('test.pkl')
print(len(X_test))
print(len(Y_test))

Data loaded from test.pkl
4232
4232


In [31]:
print(X_test[:5])
print(Y_test[:5])

['Unshaded votes and votes for Mayor Duterte goes to Mar Roxas according to some reports of ballot tests.  #AyawSaDILAW', '#NoMoreChance ', " well there's other good choices like Duterte or Poe. But both of them are still undecided, I think? :( :(", 'Nognog. Pandak. Laki sa hirap. Pero corrupt. Yan si Binay!!!', 'Ex-Binay aide turns tables on Mercado | ']
[1, 1, 0, 1, 0]
